In [3]:
!pip install openpyxl
!pip install pandas

You should consider upgrading via the 'C:\Users\gannd\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


     --------------------------------------- 11.6/11.6 MB 13.4 MB/s eta 0:00:00
     --------------------------------------- 15.8/15.8 MB 12.8 MB/s eta 0:00:00
     ------------------------------------- 505.5/505.5 KB 10.5 MB/s eta 0:00:00
     ------------------------------------- 345.4/345.4 KB 20.9 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\gannd\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
import openpyxl
import pandas as pd
import glob

In [5]:
import_file_path = 'C:\金沢大学\ゴルフ部'
excel_sheet_name = '2024年度ゴルフ部部員名簿'
export_file_path = 'C:\金沢大学\ゴルフ部\output'

In [8]:
df_order = pd.read_excel(import_file_path, sheet_name = excel_sheet_name)

PermissionError: [Errno 13] Permission denied: 'C:\\金沢大学\\ゴルフ部'

In [ ]:
df_order

In [ ]:
person_name = df_order['氏名'].unique()

In [ ]:
person_name